In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
import re
from dotenv import load_dotenv
import pandas as pd

from IPython.display import Markdown, display, HTML
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
load_dotenv()
os.chdir(os.path.dirname(os.getcwd()))

df = pd.read_parquet("./data/splade_embeds.parquet")
# Convert column names to snake_case for compatibility with LanceDB
original_columns = df.columns
snake_case_columns = {col: re.sub(r'(?<!^)(?=[A-Z])', '_', col).lower() for col in original_columns}
df.rename(columns=snake_case_columns, inplace=True)
df.head(2)

,index,created_utc,full_link,id,body,title,text_label,flair_label,embeddings,token_count,llm_title,state,kmeans_label,topic_title,splade_embeddings
0,1078,1575952538,https://www.reddit.com/r/legaladvice/comments/...,e8lsen,I applied for a job and after two interviews I...,"Failed a drug test due to amphetamines, I have...",employment,5,"[9.475638042064453e-05, 0.0005111666301983955,...",493,"""Validity of Schedule II Drug Prescription in ...",PR,8,Employment Legal Concerns and Issues,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,2098,1577442453,https://www.reddit.com/r/legaladvice/comments/...,eg9ll2,"Hi everyone, thanks in advance for any guidanc...","Speeding ticket in Tennessee, Georgia Driver's...",driving,4,"[-0.006706413111028856, 0.020911016696181495, ...",252,"""Speeding ticket consequences for out-of-state...",KY,10,Legal Topics in Traffic Violations,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [2]:
df.sort_values(by='token_count', ascending=False, inplace=True)

In [3]:
test_query = """
Is it legal for my employer to discriminating against me for being a medical marijuana patient?
"""

Markdown(test_query)


Is it legal for my employer to discriminating against me for being a medical marijuana patient?


In [4]:
from src.parsing.search import find_fuzzy_matches_in_df, find_closest_matches_with_bm25_df

In [5]:
bm25_test = find_closest_matches_with_bm25_df(
    query=test_query,
    df=df,
    k=5,
    text_column='body',
)

In [6]:
bm25_test['body'].tolist()

["I'm a New York State medical marijuana patient. I also work in healthcare. I applied to a new job at a new hospital, and they are discriminating against me for being a medical marijuana patient. I was offered the job and accepted, but when I went to get my pre-employment physical conducted, I gave them my medical marijuana card and informed them that I am a patient. They are now refusing to hire me. Is this legal? I already contacted the division of human rights at the labor department and they said I may or may not have a case.",
 'Hello,\n\nI was considering getting a medical marijuana card for my back pain. I\'ve dealt with this chronically for years and it\'s one of the few things that really help.\n\nThe problem is my company has a zero tolerance drug policy. The majority  of our employees are DOT regulated drivers so of course they are not able to able to smoke marijuana even if they have a medical card, this makes sense to me.\n\nAs for the rest of us, even us office folk, we 

In [7]:
fuzzy_test = find_fuzzy_matches_in_df(
    query='mariwanna',
    df=df,
    k=5,
    text_column='body',
    words_after=5,
    words_before=5,
)

In [8]:
fuzzy_test['context'].tolist()

['father conceal loads of drugs, marijuana and some black looking drug,',
 'years for a possession of marijuana charge. I’ve violated a couple',
 'damaged property, and constantly had marijuana in her possession on the',
 'tolerance for the use of marijuana regardless of local laws. I',
 'the roommates getting arrested for marijuana charges on the property and']

In [9]:
from src.search.query_expansion import segment

queries = segment(
    test_query,
    n='3 to 5'
)

In [10]:
queries.searches

[SubQuestion(chain_of_thought="To understand if an employer's actions are legal, we first need to examine the federal laws regarding marijuana use and employment. This will help determine if there are any overarching legal protections or prohibitions.", sub_question_topic='Federal Laws on Marijuana and Employment', sub_question_query='What are the federal laws regarding marijuana use and employment?', sub_question_keywords=['federal laws', 'marijuana', 'employment']),
 SubQuestion(chain_of_thought="Since marijuana laws can vary significantly by state, it's important to look into the specific state laws where the employment is based. This will help identify any state-specific protections for medical marijuana patients in the workplace.", sub_question_topic='State Laws on Medical Marijuana and Employment', sub_question_query='What are the state laws regarding medical marijuana use and employment?', sub_question_keywords=['state laws', 'medical marijuana', 'employment']),
 SubQuestion(cha

In [11]:
from src.agent.tools.semantic_search import SemanticSearch
from src.agent.tools.splade_search import SparseEmbeddingsSplade
from src.search.query_expansion import segment

semantic_search = SemanticSearch(
    df=df,
    embedding_col_name='embeddings'
)

splade_search = SparseEmbeddingsSplade(
    df=df,
    text_column='title',
    splade_column='splade_embeddings'
)

2024-04-06 03:49:18 - INFO - Using pre-computed 'title' embeddings from existing column: splade_embeddings


In [12]:
vector_results, keyword_results = queries.execute(semantic_search, splade_search)

2024-04-06 03:49:18 - INFO - 

Thought: To understand if an employer's actions are legal, we first need to examine the federal laws
regarding marijuana use and employment. This will help determine if there are any overarching legal
protections or prohibitions.
Search topic: Federal Laws on Marijuana and Employment
2024-04-06 03:49:18 - INFO - Running vector (OpenAI) search: What are the federal laws regarding marijuana use and employment?
2024-04-06 03:49:18 - INFO - Running keyword (SPLADE) search: federal laws, marijuana, employment
2024-04-06 03:49:18 - INFO - Returning 10 records from vector search and 10 from keywords
2024-04-06 03:49:18 - INFO - ---------------------------------------------------------------------------
2024-04-06 03:49:18 - INFO - 

Thought: Since marijuana laws can vary significantly by state, it's important to look into the specific state
laws where the employment is based. This will help identify any state-specific protections for
medical marijuana patients i

In [13]:
from src.search.doc_joiner import DocJoinerDF
from src.embedding_models.models import ColbertReranker
from src.search.threadpool import run_functions_tuples_in_parallel

In [14]:
USEFUL_PAT = "Yes useful"
NONUSEFUL_PAT = "Not useful"
CHUNK_FILTER_PROMPT = f"""
Determine if the reference section is USEFUL for answering the user query.
It is good enough for the section to be related or similar to the query, \
it should be similar information that is USEFUL for informing the query.
If the section contains ANY useful information, that is good enough, \
it does not need to fully answer the user query, and be a relevant and USEFUL reference.

Reference Section:
```
{{chunk_text}}
```

User Query:
```
{{user_query}}
```

Respond with EXACTLY AND ONLY: "{USEFUL_PAT}" or "{NONUSEFUL_PAT}"
""".strip()

In [15]:
from langchain.schema.messages import AIMessage
from langchain.schema.messages import BaseMessage
from langchain.schema.messages import HumanMessage
from langchain.schema.messages import SystemMessage

def dict_based_prompt_to_langchain_prompt(
    messages: list[dict[str, str]]
) -> list[BaseMessage]:
    prompt: list[BaseMessage] = []
    for message in messages:
        role = message.get("role")
        content = message.get("content")
        if not role:
            raise ValueError(f"Message missing `role`: {message}")
        if not content:
            raise ValueError(f"Message missing `content`: {message}")
        elif role == "user":
            prompt.append(HumanMessage(content=content))
        elif role == "system":
            prompt.append(SystemMessage(content=content))
        elif role == "assistant":
            prompt.append(AIMessage(content=content))
        else:
            raise ValueError(f"Unknown role: {role}")
    return prompt

In [16]:
from typing import Callable
from langchain_openai import ChatOpenAI

def llm_eval_chunk(query: str, chunk_content: str) -> bool:
    def _get_usefulness_messages() -> list[dict[str, str]]:
        messages = [
            {
                "role": "user",
                "content": CHUNK_FILTER_PROMPT.format(
                    chunk_text=chunk_content, user_query=query
                ),
            },
        ]

        return messages

    def _extract_usefulness(model_output: str) -> bool:
        """Default 'useful' if the LLM doesn't match pattern exactly.
        This is because it's better to trust the (re)ranking if LLM fails"""
        if model_output.content.strip().strip('"').lower() == NONUSEFUL_PAT.lower():
            return False
        return True

    llm = ChatOpenAI(model='gpt-3.5-turbo')

    messages = _get_usefulness_messages()
    filled_llm_prompt = dict_based_prompt_to_langchain_prompt(messages)
    model_output = llm.invoke(filled_llm_prompt)

    return _extract_usefulness(model_output)


def llm_batch_eval_chunks(
    query: str, chunk_contents: list[str], use_threads: bool = True
) -> list[bool]:
    if use_threads:
        functions_with_args: list[tuple[Callable, tuple]] = [
            (llm_eval_chunk, (query, chunk_content)) for chunk_content in chunk_contents
        ]

        print(
            "Running LLM usefulness eval in parallel (following logging may be out of order)"
        )
        parallel_results = run_functions_tuples_in_parallel(
            functions_with_args, allow_failures=True
        )

        # In case of failure/timeout, don't throw out the chunk
        return [True if item is None else item for item in parallel_results]

    else:
        return [
            llm_eval_chunk(query, chunk_content) for chunk_content in chunk_contents
        ]

In [17]:
from llama_index_client import TextNode
from pydantic import BaseModel
from src.search.models import dataframe_to_text_nodes

def filter_chunks(
    query: str,
    chunks_to_filter: list[BaseModel],
    max_llm_filter_chunks: int = 20,
) -> list[BaseModel]:
    """Filters chunks based on whether the LLM thought they were relevant to the query.

    Args:
        query (str): The query to filter chunks against.
        chunks_to_filter (list[TextNode]): A list of TextNode objects to filter.
        max_llm_filter_chunks (int, optional): The maximum number of chunks to consider. Defaults to 20.

    Returns:
        list[TextNode]: A list of TextNode objects that were marked as relevant.
    """
    chunks_to_filter = chunks_to_filter[: max_llm_filter_chunks]
    llm_chunk_selection = llm_batch_eval_chunks(
        query=query,
        chunk_contents=[chunk.content for chunk in chunks_to_filter],
    )
    return [
        chunk
        for ind, chunk in enumerate(chunks_to_filter)
        if llm_chunk_selection[ind]
    ]

In [18]:
from src.types import Document
from src.utils.pydantic_utils import dataframe_to_documents, dataframe_to_pydantic_objects

In [19]:
docs = dataframe_to_documents(vector_results[0], content='body', embedding='embeddings')

In [20]:
assert 1+1==3

AssertionError: 

In [21]:
docs

[DynamicDocument:
 content: 'I work a federal job. Certain positions at my work get drug tested because
 of their duties. To my knowledge, my position does not get tested. It's
 possible for anyone to get tested but it looks like they just don't test
 mine. I use CBD/hemp, which is legal in my state but not federally. I know
 I'm t'..., 
 metadata: {
     "source": "context"
 }, score: 0.8717314600944519,
 DynamicDocument:
 content: 'I have my medical marijuana card because mj lets me eat, not feel like my
 body is coming apart at it's joints, feel like life is not so hopeless, and
 sleep soundly, but I've been looking for new jobs and I'm unsure about drug
 testing.   I've looked it up and know its a state-to-state thing, but I'm'..., 
 metadata: {
     "source": "context"
 }, score: 0.8560442924499512,
 DynamicDocument:
 content: 'I'm a New York State medical marijuana patient. I also work in healthcare.
 I applied to a new job at a new hospital, and they are discriminating
 against 

In [22]:
filtered_chunks = filter_chunks(test_query, docs)
print(f"\nReturned {len(filtered_chunks)} nodes")
print(f"{test_query}\n")
for content in filtered_chunks:
    print(content)
    print("-" * 50)

Running LLM usefulness eval in parallel (following logging may be out of order)

Returned 3 nodes

Is it legal for my employer to discriminating against me for being a medical marijuana patient?


I have my medical marijuana card because mj lets me eat, not feel like my
body is coming apart at it's joints, feel like life is not so hopeless, and
sleep soundly, but I've been looking for new jobs and I'm unsure about drug
testing.   I've looked it up and know its a state-to-state thing, but I'm
{
    "source": "context"
}
--------------------------------------------------
I'm a New York State medical marijuana patient. I also work in healthcare.
I applied to a new job at a new hospital, and they are discriminating
against me for being a medical marijuana patient. I was offered the job and
accepted, but when I went to get my pre-employment physical conducted, I
gave t
{
    "source": "context"
}
--------------------------------------------------
Hello,  I was considering getting a medical 

In [23]:
print(filtered_chunks[0])

I have my medical marijuana card because mj lets me eat, not feel like my
body is coming apart at it's joints, feel like life is not so hopeless, and
sleep soundly, but I've been looking for new jobs and I'm unsure about drug
testing.   I've looked it up and know its a state-to-state thing, but I'm
{
    "source": "context"
}


In [ ]:
cols = df.columns.tolist()
cols.remove('embeddings')

i = 0

query = queries.searches[i].sub_question_keywords
nodes = dataframe_to_text_nodes(
    keyword_results[i],
    id_column='index', 
    text_col='body', 
    metadata_fields=cols
)

In [ ]:
from src.search.models import text_nodes_to_dataframe

node_df = text_nodes_to_dataframe(nodes)
node_df.head(1)

,text,index,created_utc,full_link,id,body,title,text_label,flair_label,token_count,llm_title,state,kmeans_label,topic_title,splade_embeddings
0,I'm a New York State medical marijuana patient...,8075,1590524169,https://www.reddit.com/r/legaladvice/comments/...,gr4q8s,I'm a New York State medical marijuana patient...,Is it legal in New York to discriminate agains...,employment,5,114,"""Can a hospital refuse to hire a medical marij...",MI,8,Employment Legal Concerns and Issues,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [ ]:
filtered_chunks = filter_chunks(test_query, nodes)
print(f"\nReturned {len(filtered_chunks)} nodes")
print(f"{query}\n")
for obj in filtered_chunks:
    print(obj)
    print("-" * 50)


Running LLM usefulness eval in parallel (following logging may be out of order)

Returned 0 nodes
['medical marijuana', 'legal status', 'state laws']



In [24]:
df_merger = DocJoinerDF(join_mode="merge", top_k=20)

In [25]:
vector_res = df_merger.run(vector_results, text_column='body')
keyword_res = df_merger.run(keyword_results, text_column='title')
vector_res.shape, keyword_res.shape

((18, 18), (20, 18))

In [26]:
reranker = ColbertReranker(column='body')

In [27]:
vec_res = vector_res.drop(columns=['embeddings', 'splade_embeddings'])
kw_res = keyword_res.drop(columns=['embeddings', 'splade_embeddings'])

In [28]:
hybrid_results = reranker.rerank_hybrid(test_query, vec_res, kw_res)

In [29]:
hybrid_results.drop_duplicates(subset=['body'], keep='last', inplace=True)

In [30]:
hybrid_results.shape

(27, 16)

In [31]:
from IPython.display import Markdown

# Convert the 'body' column of the first 5 rows into a Markdown formatted string
markdown_text = '\n\n---\n\n'.join(hybrid_results.head()['body'].tolist())
display(Markdown(markdown_text))

I'm a New York State medical marijuana patient. I also work in healthcare. I applied to a new job at a new hospital, and they are discriminating against me for being a medical marijuana patient. I was offered the job and accepted, but when I went to get my pre-employment physical conducted, I gave them my medical marijuana card and informed them that I am a patient. They are now refusing to hire me. Is this legal? I already contacted the division of human rights at the labor department and they said I may or may not have a case.

---

Hello,

I was considering getting a medical marijuana card for my back pain. I've dealt with this chronically for years and it's one of the few things that really help.

The problem is my company has a zero tolerance drug policy. The majority  of our employees are DOT regulated drivers so of course they are not able to able to smoke marijuana even if they have a medical card, this makes sense to me.

As for the rest of us, even us office folk, we are all still subject to randoms.  Basically the logic is that everyone is considered to be in a "safety sensitive" position because we can drive on company time or drive a company vehicle. We have a memo in our drug and alcohol handbook to "all employees" stating that just because some states have legalized marijuana, does not mean we can disregard company policy. It does NOT say anything about medical cards.

I live in Arizona where medical marijuana is legal. Is my company able to trump me having a medical card because of their policy?  If I had a random and told the tester  I had a medical card, would it violate HIPAA for the MRO to inform my boss that I tested positive and have a medical card? What can I do? 

I feel really defeated by my pain and I just wish I could do the one thing that helps me.

Thank you.

---

This is in Iowa. My employer recently initiated a random drug testing program. This is not good news for me as I am an occasional recreational user of marijuana. I began researching my rights as far as the drug testing and came across Iowa Code 730.5. It says that employees have the right to testing done with "regard to the privacy of the employee." 

My question is, what does this "regard to the privacy of the employee" mean? Are they allowed to watch me urinate while taking the test? In a previous round of drug tests they did, my co-worker told me that a police officer and one of the employees of the company doing the testing were in the bathroom while they were told to urinate. Is this legal in the state of Iowa? 

Any insight appreciated, thanks 

EDIT: I just found out that the guy watching was a  "former" police officer who now works at the drug testing place. Not an actual police officer. Sorry for confusion. 

---

I have my medical marijuana card because mj lets me eat, not feel like my body is coming apart at it's joints, feel like life is not so hopeless, and sleep soundly, but I've been looking for new jobs and I'm unsure about drug testing. 

I've looked it up and know its a state-to-state thing, but I'm still a little iffy about Pennsylvania and how weirdly 50/50 it seems to be with medical marijuana. I don't smoke before or during work and never intend to do so. However I'm unsure about the legality of workers compensation and drug testing during a hire situation if I happen to fail the test. 

Will I get fired or fined from work and would the employer looking to hire me have grounds to dismiss me?

Also, this is my first time on this sub so I hope this isn't an annoying question or something.

---

I recently started working at a small manufacturing company in Los Angeles working as an e-commerce specialist. My boss told me that there would be random drug testing for employees. I’ve been looking up the legality of random drug tests in California and to my best understanding random drug tests are only allowed for employees who work more dangerous jobs, and I do not qualify under that standard. Would a random drug test for me be legal? 

Also, if I were to refuse to take the random drug test could I be fired?

In [32]:
hybrid_results['search_type'].value_counts()

search_type
vector    14
splade    13
Name: count, dtype: int64

In [33]:
from src.agent.tools.utils import aget_fact_patterns_df
import nest_asyncio
nest_asyncio.apply()

prep_df = await aget_fact_patterns_df(hybrid_results, 'body', "id")

Processing summaries:   0%|          | 0/27 [00:00<?, ?it/s]

Processing summaries: 100%|██████████| 27/27 [00:07<00:00,  3.79it/s]


In [34]:
prep_df.head(2)

,index,created_utc,full_link,id,body,title,text_label,flair_label,token_count,llm_title,state,kmeans_label,topic_title,clean_text,search_type,_relevance_score,summary
0,8075,1590524169,https://www.reddit.com/r/legaladvice/comments/...,36598d2ca979f02397b530a5c36d7a2a,I'm a New York State medical marijuana patient...,Is it legal in New York to discriminate agains...,employment,5,114,"""Can a hospital refuse to hire a medical marij...",MI,8,Employment Legal Concerns and Issues,new york state medical marijuana patient also ...,splade,0.821348,"A New York State medical marijuana patient, wh..."
1,7331,1576018396,https://www.reddit.com/r/legaladvice/comments/...,b20c68f342dceb310b1c8190753aa3af,"Hello,\n\nI was considering getting a medical ...",Considering getting a medical marijuana card- ...,employment,5,276,"""Legal implications of company drug policies a...",PA,8,Employment Legal Concerns and Issues,hello considering getting medical marijuana ca...,vector,0.669316,An individual considering obtaining a medical ...


In [35]:
from src.agent.tools.utils import create_formatted_input, get_final_answer

formatted_input = create_formatted_input(
    prep_df, test_query, context_token_limit=8000
    )

response_model = get_final_answer(formatted_input, model_name="gpt-4-turbo-preview")

In [36]:
Markdown(response_model.research_report)

The query raises the issue of employment-related discrimination based on an individual's status as a medical marijuana patient. This concern intersects with several laws and regulations, including but not limited to, employment discrimination laws, medical marijuana laws at the state level, and policies regarding drug testing in the workplace. 

The most directly relevant case is the first search result. This individual was offered a job in New York, which was later rescinded upon disclosure of their medical marijuana patient status during a pre-employment physical. They reached out to the division of human rights at the labor department for guidance, indicating a potential legal issue surrounding employment discrimination based on medical marijuana use [[1](https://www.reddit.com/r/legaladvice/comments/gr4q8s/is_it_legal_in_new_york_to_discriminate_against/)].

Another pertinent case comes from Arizona, where an individual debated the ramifications of obtaining a medical marijuana card on their employment due to their company's zero-tolerance drug policy [[2](https://www.reddit.com/r/legaladvice/comments/e8y7gr/considering_getting_a_medical_marijuana_card/)]. This scenario mirrors the query by highlighting the tension between state-legalized medical marijuana and employer drug policies. 

The situation described in Pennsylvania also sheds light on this issue. The individual, a medical marijuana cardholder, expressed concerns about drug testing during the hiring process and potential repercussions upon failing the test due to their legitimate medical use of marijuana [[4](https://www.reddit.com/r/legaladvice/comments/gnrqby/if_i_have_a_medical_marijuana_card_can_i_be_fired/)].

Finally, the scenario in New Jersey presents a scenario where prescribed medication (though not specifically marijuana) constituted grounds for a job offer being rescinded, which parallels concerns about discrimination based on medical treatments or prescriptions [[6](https://www.reddit.com/r/legaladvice/comments/gnpjof/nj_i_was_denied_a_job_due_to_being_prescribed/)].

These cases collectively illustrate the complexity of navigating employment and medical marijuana use. They suggest a delicate balance between complying with state laws that permit medical marijuana and adhering to employer drug policies that may not differentiate between recreational and medical use. The legal landscape around this issue appears to be evolving, and the specifics may vary significantly depending on jurisdiction, making it imperative to consider local laws and regulations alongside federal standards when assessing legality.

In [ ]:
assert 1 + 1 == 3

AssertionError: 

In [ ]:
from src.search.query_expansion import generate_subquestions

questions = generate_subquestions(test_query, n='any number of')
questions.questions

['Should I just pay the rent and wait for my refund?',
 'If the post office shows that my original money order was cashed, am I out that money?',
 'What can I do about a landlord who is slow to make repairs?',
 'Is it legal for my landlord to enter my apartment without permission?',
 'Can a landlord raise rent in response to making repairs?',
 'How do I handle disruptive neighbors?',
 'What should I do if I suspect my landlord of stealing from me?',
 'What are my rights as a tenant in Missouri?',
 'Is it legal to use pliers to turn on water in lieu of a broken knob?',
 "What actions can I take if I've been treated unfairly by my landlord?"]

In [ ]:
questions._raw_response.usage

CompletionUsage(completion_tokens=147, prompt_tokens=1130, total_tokens=1277)

In [ ]:
from src.search.query_filter import generate_query_plan, auto_filter_fts_search

In [ ]:
query_plan = generate_query_plan(
    input_df=df,
    query=test_query,
    filter_fields=[
        'state',
    ]
)
filtered_df = query_plan.filter_df(df=df)

2024-03-19 21:42:45 - INFO - Schema shown to LLM: 
Name of each field, its type and unique values (up to 20):
* state (string);  Values - ['NM' 'IN' 'WY' 'NH' 'MP' 'PA' 'MH' 'ID' 'AR' 'MA' 'KS' 'AS' 'ND' 'PR'
 'DE' 'FL' 'LA' 'OR' 'VT' 'PW'], ... 39 more
        


2024-03-19 21:42:55 - INFO - Input DataFrame has 5,000 rows
2024-03-19 21:42:55 - INFO - Applying filter(s): state LIKE '%OR%'
2024-03-19 21:43:03 - INFO - Filtered DataFrame has 86 rows


In [ ]:
filtered_df.head(2)

,index,created_utc,full_link,id,body,title,text_label,flair_label,embeddings,token_count,llm_title,state,kmeans_label,topic_title,splade_embeddings
0,2029,1578267399,https://www.reddit.com/r/legaladvice/comments/...,ekl2ef,For context I live in the Philippines. I wont ...,My professor refuses to show us ALL of our tes...,school,9,"[-0.00954271624451715, 0.007157037183387862, 0...",953,"""Unrevealed Grades and Lack of Transparency: S...",OR,9,Legal Consequences of False Accusations,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,3320,1591126549,https://www.reddit.com/r/legaladvice/comments/...,gve3nq,Edit: I live in Washington state.\n\nSo I live...,My landlord has been harassing me about my pet...,housing,7,"[-0.0034782202413053045, 0.00616729225832095, ...",759,"""Legal dispute over pet snake: Landlord threat...",OR,3,Rental Property and Landlord Matters,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [ ]:
print(query_plan.original_query)
print(query_plan.rephrased_query)

Do I have any legal recourse here? I know Oregon is an 'at will' state, but it sounds like there are at LEAST two instances that offer grounds for wrongful termination (just based on my limited knowledge of the ADA, dept of labor, BOLI, etc.). 
legal recourse for wrongful termination in 'at will' employment including issues related to mistreatment, health code violations, improper handling of company money, and potential discrimination due to medical conditions


In [ ]:
test_res = auto_filter_fts_search(
    df=df,
    query='marijuana',
    top_k=20,
    text_column="body",
    embeddings_column="embeddings",
    filter_fields=[
        'state',
    ])

2024-03-19 21:44:19 - INFO - Schema shown to LLM: 
Name of each field, its type and unique values (up to 20):
* state (string);  Values - ['NM' 'IN' 'WY' 'NH' 'MP' 'PA' 'MH' 'ID' 'AR' 'MA' 'KS' 'AS' 'ND' 'PR'
 'DE' 'FL' 'LA' 'OR' 'VT' 'PW'], ... 39 more
        


2024-03-19 21:44:22 - INFO - No filters were identified for query: marijuana
2024-03-19 21:44:22 - INFO - Revised query: marijuana
2024-03-19 21:44:23 - INFO - Full Text Search (FTS) search yielded a DataFrame with 20 rows


In [ ]:
Markdown(test_res['body'].tolist()[0])

I'm a New York State medical marijuana patient. I also work in healthcare. I applied to a new job at a new hospital, and they are discriminating against me for being a medical marijuana patient. I was offered the job and accepted, but when I went to get my pre-employment physical conducted, I gave them my medical marijuana card and informed them that I am a patient. They are now refusing to hire me. Is this legal? I already contacted the division of human rights at the labor department and they said I may or may not have a case.